In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

from src.folder_handler import *
from src.cort_processor import *
from src.tdt_support import *
from src.plotter import *
from src.decoders import *
import pickle

# load data
data should be a single folder. Inside there should be a 'angles' (with angles), 'pose-3d', and 'tdt' folders named exactly that

In [2]:
session512 = CortProcessor('/home/diya/Documents/rat-fes/data/rat-annie/5-12') #insert name of your folder here

read from t=0s to t=186.82s
read from t=0s to t=164.93s
read from t=0s to t=222.7s
read from t=0s to t=241.72s


# process data
here, we crop our data, turn our raw data into rates, and we resample our angles to be same times as binned spikes

In [3]:
rates, kins = session512.process()

# alternative way to load data: using pickles (and process_cort.py in scripts)

in order to process data using command line, navigate to /scripts, and run 'python3 process_cort.py *path_to_folder'

In [ ]:
with open('/home/diya/Documents/rat-fes/data/pickles/rat-annie_5-12_session.pkl', 'rb') as inp:
    session512 = pickle.load(inp)

# other pipeline processing, like getting toe height

In [ ]:
toe_height = session512.process_toe_height()

In [ ]:
session512.data['toe_height']

# and now some decoding

## angles

In [ ]:
h_angles, vaf_angles, best_x_angles, best_y_angles = session512.decode_angles()

In [ ]:
average_vaf_angle = np.average(vaf_angles, 1)
print(average_vaf_angle)

In [ ]:
session512.data['angle_names']

In [ ]:
plot_wiener_filter_predic(best_x_angles, best_y_angles[:,3], h_angles[:,3])

## toe_height

In [ ]:
h_toe, vaf_toe, best_x_toe, best_y_toe = session512.decode_toe_height()

In [ ]:
average_toe_vaf = np.average(vaf_toe)
print(average_toe_vaf)

In [ ]:
plot_wiener_filter_predic(best_x_toe, best_y_toe, h_toe)

# or dividing into gaits, then plotting sorted raster.

In [ ]:
gait_indices, avg_gait_samples = session512.get_gait_indices()

In [ ]:
X_gait, Y_gait = session512.divide_into_gaits()

In [ ]:
#proc_rates, proc_angles = session512.remove_bad_gaits()

In [ ]:
df = session512.neuron_tuning()

In [ ]:
plot_raster(df)